In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [5]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
training_sentences

['Hi',
 'How are you',
 'Is anyone there?',
 'Hello',
 'Good day',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'What hours are you open?',
 'What are your hours?',
 'When are you open?',
 'What is your location?',
 'Where are you located?',
 'What is your address?',
 'Where is your restaurant situated?',
 'Do you take credit cards?',
 'Do you accept Mastercard?',
 'Are you cash only?',
 'What is your menu for today?',
 'What are you serving today?',
 "What is today's special?",
 'Do you provide home delivery?',
 'Do you deliver the food?',
 'What are the home delivery options?',
 'What is your Menu?',
 'What are the main course options?',
 'Can you tell me the most delicious dish from the menu?',
 "What is the today's special?"]

In [7]:
labels

['greeting',
 'goodbye',
 'thanks',
 'hours',
 'location',
 'payments',
 'todaysmenu',
 'deliveryoption',
 'menu']

In [8]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'hours',
 'hours',
 'hours',
 'location',
 'location',
 'location',
 'location',
 'payments',
 'payments',
 'payments',
 'todaysmenu',
 'todaysmenu',
 'todaysmenu',
 'deliveryoption',
 'deliveryoption',
 'deliveryoption',
 'menu',
 'menu',
 'menu',
 'menu']

In [9]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [10]:
training_labels

array([2, 2, 2, 2, 2, 1, 1, 1, 7, 7, 7, 3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 8,
       8, 8, 0, 0, 0, 5, 5, 5, 5], dtype=int64)

In [11]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 153       
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 2.1982 - accuracy: 0.0968
Epoch 2/500
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 2.1407 - accuracy: 0.1290
Epoch 74/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1385 - accuracy: 0.1290
Epoch 75/500
1/1 [==============================] - 0s 5ms/step - loss: 2.1363 - accuracy: 0.1290
Epoch 76/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1340 - accuracy: 0.1290
Epoch 77/500
1/1 [==============================] - 0s 5ms/step - loss: 2.1316 - accuracy: 0.1290
Epoch 78/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1292 - accuracy: 0.1290
Epoch 79/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1268 - accuracy: 0.1290
Epoch 80/500
1/1 [==============================] - 0s 5ms/step - loss: 2.1246 - accuracy: 0.1290
Epoch 81/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1223 - accuracy: 0.1290
Epoch 82/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1200 - accuracy: 0.1290
Epoch 83/500
1/1 [===============

1/1 [==============================] - 0s 5ms/step - loss: 1.8527 - accuracy: 0.1290
Epoch 157/500
1/1 [==============================] - 0s 5ms/step - loss: 1.8488 - accuracy: 0.1290
Epoch 158/500
1/1 [==============================] - 0s 5ms/step - loss: 1.8447 - accuracy: 0.1290
Epoch 159/500
1/1 [==============================] - 0s 4ms/step - loss: 1.8411 - accuracy: 0.1290
Epoch 160/500
1/1 [==============================] - 0s 5ms/step - loss: 1.8370 - accuracy: 0.1290
Epoch 161/500
1/1 [==============================] - 0s 4ms/step - loss: 1.8330 - accuracy: 0.1290
Epoch 162/500
1/1 [==============================] - 0s 5ms/step - loss: 1.8293 - accuracy: 0.1290
Epoch 163/500
1/1 [==============================] - 0s 4ms/step - loss: 1.8251 - accuracy: 0.1290
Epoch 164/500
1/1 [==============================] - 0s 5ms/step - loss: 1.8211 - accuracy: 0.1290
Epoch 165/500
1/1 [==============================] - 0s 4ms/step - loss: 1.8172 - accuracy: 0.1290
Epoch 166/500
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 1.4343 - accuracy: 0.5484
Epoch 239/500
1/1 [==============================] - 0s 4ms/step - loss: 1.4285 - accuracy: 0.5806
Epoch 240/500
1/1 [==============================] - 0s 4ms/step - loss: 1.4226 - accuracy: 0.5806
Epoch 241/500
1/1 [==============================] - 0s 5ms/step - loss: 1.4166 - accuracy: 0.5806
Epoch 242/500
1/1 [==============================] - 0s 4ms/step - loss: 1.4109 - accuracy: 0.5806
Epoch 243/500
1/1 [==============================] - 0s 5ms/step - loss: 1.4049 - accuracy: 0.5806
Epoch 244/500
1/1 [==============================] - 0s 4ms/step - loss: 1.3990 - accuracy: 0.5806
Epoch 245/500
1/1 [==============================] - 0s 5ms/step - loss: 1.3931 - accuracy: 0.5806
Epoch 246/500
1/1 [==============================] - 0s 5ms/step - loss: 1.3873 - accuracy: 0.5806
Epoch 247/500
1/1 [==============================] - 0s 5ms/step - loss: 1.3814 - accuracy: 0.5806
Epoch 248/500
1/1 [=====

1/1 [==============================] - 0s 5ms/step - loss: 1.0080 - accuracy: 0.6452
Epoch 321/500
1/1 [==============================] - ETA: 0s - loss: 1.0037 - accuracy: 0.64 - 0s 8ms/step - loss: 1.0037 - accuracy: 0.6452
Epoch 322/500
1/1 [==============================] - 0s 7ms/step - loss: 0.9992 - accuracy: 0.6452
Epoch 323/500
1/1 [==============================] - 0s 6ms/step - loss: 0.9948 - accuracy: 0.6129
Epoch 324/500
1/1 [==============================] - 0s 5ms/step - loss: 0.9903 - accuracy: 0.6129
Epoch 325/500
1/1 [==============================] - 0s 4ms/step - loss: 0.9859 - accuracy: 0.6452
Epoch 326/500
1/1 [==============================] - 0s 5ms/step - loss: 0.9818 - accuracy: 0.6129
Epoch 327/500
1/1 [==============================] - 0s 4ms/step - loss: 0.9776 - accuracy: 0.6129
Epoch 328/500
1/1 [==============================] - 0s 4ms/step - loss: 0.9734 - accuracy: 0.6129
Epoch 329/500
1/1 [==============================] - 0s 5ms/step - loss: 0.9693 -

1/1 [==============================] - 0s 5ms/step - loss: 0.7115 - accuracy: 0.8387
Epoch 403/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7079 - accuracy: 0.8710
Epoch 404/500
1/1 [==============================] - 0s 6ms/step - loss: 0.7043 - accuracy: 0.8710
Epoch 405/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7005 - accuracy: 0.8710
Epoch 406/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6969 - accuracy: 0.8710
Epoch 407/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6934 - accuracy: 0.8710
Epoch 408/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6896 - accuracy: 0.8710
Epoch 409/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6858 - accuracy: 0.8710
Epoch 410/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6821 - accuracy: 0.8710
Epoch 411/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6783 - accuracy: 0.8710
Epoch 412/500
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 0.3919 - accuracy: 0.9032
Epoch 485/500
1/1 [==============================] - 0s 6ms/step - loss: 0.3879 - accuracy: 0.9032
Epoch 486/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3844 - accuracy: 0.9032
Epoch 487/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3808 - accuracy: 0.9032
Epoch 488/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3770 - accuracy: 0.9032
Epoch 489/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3734 - accuracy: 0.9032
Epoch 490/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3698 - accuracy: 0.9032
Epoch 491/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3663 - accuracy: 0.9032
Epoch 492/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3627 - accuracy: 0.9032
Epoch 493/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3592 - accuracy: 0.9032
Epoch 494/500
1/1 [=====

In [13]:
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: 